In [ ]:
from sklearn.model_selection import GridSearchCV
import cv2 
import numpy as np
import os
from matplotlib import pyplot as plt
from func_hair import smplextract_hair
from PIL import Image
from sklearn.metrics import jaccard_score
import pdb
from skimage.feature import graycomatrix, graycoprops
from sklearn.model_selection import train_test_split
from pyfeats import fos
import pandas as pd
from skimage import filters
from scipy.stats import skew
from scipy.stats import kurtosis
from skimage.feature import local_binary_pattern
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import classification_report
import pickle
from sklearn.utils.estimator_checks import check_estimator
from sklearn.linear_model import SGDClassifier
from sklearn import svm
from scipy import interp
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import csv
header = ['mean_R', 'std_R', 'skew_R', 'mean_G',  'std_G',  'skew_G',  'mean_B',  'std_B',  'skew_B',   
                 'mean_H', 'std_H', 'skew_H', 'mean_S',  'std_S',  'skew_S',  'mean_V',  'std_V', 'skew_V',              
                 'CM_black', 'CM_red', 'CM_bluegray', 'CM_white', 'CM_lightbrown', 'CM_darkbrown' 
               
                 'mean_ccR', 'std_ccR', 'skew_ccR', 'mean_ccG',  'std_ccG',  'skew_ccG',  'mean_ccB',  'std_ccB',  'skew_ccB', 
                 'mean_ccH', 'std_ccH', 'skew_ccH', 'mean_ccS',  'std_ccS',  'skew_ccS',  'mean_ccV',  'std_ccV',  'skew_ccV',
                 'mean_mxR', 'std_mxR', 'skew_mxR', 'mean_mxG',  'std_mxG',  'skew_mxG',  'mean_mxB',  'std_mxB',  'skew_mxB', 
                 'mean_mxH', 'std_mxH', 'skew_mxH', 'mean_mxS',  'std_mxS',  'skew_mxS',  'mean_mxV',  'std_mxV',  'skew_mxV',
                 'contrast0','contrast1','contrast2','dissimilarity0','dissimilarity1','dissimilarity2','correlation0','correlation1','correlation2','homogeneity0','homogeneity1','homogeneity2',
                 'contrast0h','contrast1s','contrast2v','dissimilarity0h','dissimilarity1s','dissimilarity2v','correlation0h','correlation1s','correlation2v','homogeneity0h','homogeneity1s','homogeneity2v']
with open('features.csv1','w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(header)

pathFolder_train='/home/nisma/data1/'
train_Array = [x for x in sorted(os.listdir(pathFolder_train)) if os.path.isfile(os.path.join(pathFolder_train,x))]
outputFolder_path='/home/nisma/save2/'

def segmentation(img_o):
    h1,w1,c1=img_o.shape
   
    resized_down = cv2.resize(img_o, (256,192), interpolation= cv2.INTER_AREA)
   
    img_r= smplextract_hair(resized_down)
   

    median = cv2.medianBlur(img_r,5) # Apply Median filter
    # =============================================================================
    #  

    Z = median.reshape((-1,3))

    # convert to np.float32
    Z = np.float32(Z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
    K = 8
    flags=cv2.KMEANS_RANDOM_CENTERS
    ret,label,center=cv2.kmeans(Z,K,None,criteria,10,flags)

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    kmeans_img = res.reshape((img_r.shape))
    # =============================================================================
    # =============================================================================
    #    Adaptive histogram equalization  
    # =============================================================================
    clahe = cv2.createCLAHE(clipLimit=3., tileGridSize=(8,8))

    hsv = cv2.cvtColor(kmeans_img, cv2.COLOR_BGR2HSV)# convert from BGR to HSV color space
    
     
    h, s, v = cv2.split(hsv)  # split on 3 different channels
    #apply CLAHE to the L-channel
    h1 = clahe.apply(h)
    s1 = clahe.apply(s)
    v1 = clahe.apply(v)
    lab = cv2.merge((h1,s1,v1))  # merge channels
    Enhance_img= cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR
     
    # =============================================================================
    #    making the mask for grabcut
    # =============================================================================
    hsv = cv2.cvtColor(Enhance_img, cv2.COLOR_BGR2HSV)    
    lower_green = np.array([50,100,100])
    upper_green = np.array([150,255,255])
    #lower_green = np.array([150,100,100])
    #upper_green = np.array([150,250,255])
    #mask_g = cv2.inRange(hsv, lower_green, upper_green)
    #cv2.imshow("1ramask",mask_g)
    mask_g = cv2.inRange(hsv, lower_green, upper_green)     
    ret,inv_mask = cv2.threshold(mask_g,127,255,cv2.THRESH_BINARY_INV)     
    res = cv2.bitwise_and(img_r,img_r, mask= mask_g)
    mask = np.zeros(img_r.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)
   
    # =============================================================================
    #      GRABCUT
    # =============================================================================
    if (np.sum(inv_mask[:])<8507940):
        newmask = inv_mask

        # wherever it is marked white (sure foreground), change mask=1
        # wherever it is marked black (sure background), change mask=0
        mask[newmask == 0] = 0
        mask[newmask == 255] = 1
        #dim2= cv2.grabCut(lab,mask,None,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_MASK)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        GrabCut_img2 = img_r*mask2[:,:,np.newaxis]
        
    else:
        s = (img_r.shape[0] / 10, img_r.shape[1] / 10)
        rect = (int(s[0]), int(s[1]), int(img_r.shape[0]) -int((3/10)) * int(s[0]), int(img_r.shape[1]) - int(s[1]))
        cv2.grabCut(lab,mask,rect,bgdModel,fgdModel,10,cv2.GC_INIT_WITH_RECT)
        mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
        GrabCut_img2= img_r*mask2[:,:,np.newaxis]
        
     

    # =============================================================================
    # Binarization
    # =============================================================================
    imgmask2 = cv2.medianBlur(GrabCut_img2,5)
    ret,Segmented_mask2 = cv2.threshold(imgmask2,0,255,cv2.THRESH_BINARY)
    print("segggg", Segmented_mask2.shape)
    ##cv2.imshow("seg",Segmented_mask2)
    return Segmented_mask2,img_r
    # =================================================

### Color Features
def color_moments(image, channel=3):
    if (channel==3):        
        mean_0 = np.mean(image[:,:,0])
        mean_1 = np.mean(image[:,:,1])
        mean_2 = np.mean(image[:,:,2])
        std_0  = np.std(image[:,:,0])
        std_1  = np.std(image[:,:,1])
        std_2  = np.std(image[:,:,2])
        skew_0 = skew(image[:,:,0].reshape(-1))
        skew_1 = skew(image[:,:,1].reshape(-1))
        skew_2 = skew(image[:,:,2].reshape(-1))
        kurt_0 = kurtosis(image[:,:,0].reshape(-1))
        kurt_1 = kurtosis(image[:,:,1].reshape(-1))
        kurt_2 = kurtosis(image[:,:,2].reshape(-1))
        return mean_0, std_0, skew_0, kurt_0, mean_1, std_1, skew_1, kurt_1, mean_2, std_2, skew_2, kurt_2

    
    else:
        assert False, "ERROR: The function supports only 1 or 3-channel image formats."
    


    
def GLCM(image, channel=3, bit_depth=8):
    if (channel==3):  
        GLCM_0  = graycomatrix(image[:,:,0],  [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=2**bit_depth)
        GLCM_1  = graycomatrix(image[:,:,1],  [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=2**bit_depth)
        GLCM_2  = graycomatrix(image[:,:,2],  [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], levels=2**bit_depth)
        contrast_0  = graycoprops(GLCM_0,  'contrast').mean()
        contrast_1  = graycoprops(GLCM_1,  'contrast').mean()
        contrast_2  = graycoprops(GLCM_2,  'contrast').mean()
        dissim_0    = graycoprops(GLCM_0,  'dissimilarity').mean()
        dissim_1    = graycoprops(GLCM_1,  'dissimilarity').mean()
        dissim_2    = graycoprops(GLCM_2,  'dissimilarity').mean()     
        correl_0    = graycoprops(GLCM_0,  'correlation').mean()
        correl_1    = graycoprops(GLCM_1,  'correlation').mean()
        correl_2    = graycoprops(GLCM_2,  'correlation').mean()
        homo_0      = graycoprops(GLCM_0,  'homogeneity').mean()
        homo_1      = graycoprops(GLCM_1,  'homogeneity').mean()
        homo_2      = graycoprops(GLCM_2,  'homogeneity').mean()
        return [ contrast_0, dissim_0, correl_0, homo_0, contrast_1, dissim_1,
                 correl_1, homo_1, contrast_2, dissim_2, correl_2, homo_2 ]
    
    else:
        assert False, "ERROR: The function supports only 1 or 3-channel image formats."




def melanoma_color_markers(image,mask):
    CM_black      = np.count_nonzero(((image[:,:,0].astype(float)/255)<0.20)
                                    &((image[:,:,1].astype(float)/255)<0.20)
                                    &((image[:,:,2].astype(float)/255)<0.20))*(100/np.sum(mask))
    CM_red        = np.count_nonzero(((image[:,:,0].astype(float)/255)>0.80)
                                    &((image[:,:,1].astype(float)/255)<0.20)
                                    &((image[:,:,2].astype(float)/255)<0.20))*(100/np.sum(mask))
    CM_bluegray   = np.count_nonzero(((image[:,:,0].astype(float)/255)<0.20)
                                    &((image[:,:,1].astype(float)/255)<0.72)
                                    &((image[:,:,1].astype(float)/255)>0.30)
                                    &((image[:,:,2].astype(float)/255)<0.74)
                                    &((image[:,:,2].astype(float)/255)>0.34))*(100/np.sum(mask))
    CM_white      = np.count_nonzero(((image[:,:,0].astype(float)/255)>0.80)
                                    &((image[:,:,1].astype(float)/255)>0.80)
                                    &((image[:,:,2].astype(float)/255)>0.80))*(100/np.sum(mask))
    CM_lightbrown = np.count_nonzero(((image[:,:,0].astype(float)/255)<1.00)
                                    &((image[:,:,0].astype(float)/255)>0.60)
                                    &((image[:,:,1].astype(float)/255)<0.72)
                                    &((image[:,:,1].astype(float)/255)>0.32)
                                    &((image[:,:,2].astype(float)/255)<0.45)
                                    &((image[:,:,2].astype(float)/255)>0.05))*(100/np.sum(mask))
    CM_darkbrown  = np.count_nonzero(((image[:,:,0].astype(float)/255)<0.60)
                                    &((image[:,:,0].astype(float)/255)>0.20)
                                    &((image[:,:,1].astype(float)/255)<0.46)
                                    &((image[:,:,1].astype(float)/255)>0.06)
                                    &((image[:,:,2].astype(float)/255)<0.33))*(100/np.sum(mask))
    return CM_black, CM_red, CM_bluegray, CM_white, CM_lightbrown, CM_darkbrown




def LBP(image, channel=3, P=8, R=2, bins=10):
    if (channel==3):
        lbp       = local_binary_pattern(image[:,:,0], P=P, R=R, method="uniform")
        lbp_0, _  = np.histogram(lbp, density=True, bins=bins, range=(0,int(lbp.max()+1)))
        lbp       = local_binary_pattern(image[:,:,1], P=P, R=R, method="uniform")
        lbp_1, _  = np.histogram(lbp, density=True, bins=bins, range=(0,int(lbp.max()+1)))
        lbp       = local_binary_pattern(image[:,:,2], P=P, R=R, method="uniform")
        lbp_2, _  = np.histogram(lbp, density=True, bins=bins, range=(0,int(lbp.max()+1)))
        return lbp_0, lbp_1, lbp_2

    else:
        assert False, "ERROR: The function supports only 1 or 3-channel image formats."

        
#############################################################################################################################################################
### Feature Extraction
def grey_edge(image, njet=0, mink_norm=1, sigma=1):

    gauss_image = filters.gaussian(image, sigma=sigma, multichannel=True)

    if njet == 0:
        deriv_image = [gauss_image[:, :, channel] for channel in range(3)]
    else:   
        if njet == 1:
            deriv_filter = filters.sobel
        elif njet == 2:
            deriv_filter = filters.laplace
        else:
            raise ValueError("njet should be in range[0-2]! Given value is: " + str(njet))     
        deriv_image = [np.abs(deriv_filter(gauss_image[:, :, channel])) for channel in range(3)]

    for channel in range(3):
        deriv_image[channel][image[:, :, channel] >= 255] = 0.

    if mink_norm == -1:  
        estimating_func = np.max 
    else:
        estimating_func = lambda x: np.power(np.sum(np.power(x, mink_norm)), 1 / mink_norm)
    illum = [estimating_func(channel) for channel in deriv_image]
    som   = np.sqrt(np.sum(np.power(illum, 2)))
    illum = np.divide(illum, som)

    return illum
def correct_image(image, illum):
    """
    Corrects image colors by performing diagonal transformation according to 
    given estimated illumination of the image.
    
    :param image: rgb input image (NxMx3)
    :param illum: estimated illumination of the image
    :return: corrected image
    """
    correcting_illum = illum * np.sqrt(3)
    corrected_image = image / 255.
    for channel in range(3):
        corrected_image[:, :, channel] /= correcting_illum[channel]
    return np.clip(corrected_image, 0., 1.)
def create_circular_mask(h, w, center=None, radius=None):
    if center is None: # use the middle of the image
        center = [int(w/2), int(h/2)]
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    return mask
def color_constant(img):
    # Extract Color Channels
    img_R = img[:,:,0]
    img_G = img[:,:,1]
    img_B = img[:,:,2]    
    
    # Calculate Channel Averages
    avg_R = np.mean(img_R)
    avg_G = np.mean(img_G)
    avg_B = np.mean(img_B)
    avg_all = np.mean(img)
    
    # Calculate Scaling Factor for White-Balance
    scale_R = (avg_all / avg_R)
    scale_G = (avg_all / avg_G)
    scale_B = (avg_all / avg_B)
    
    # Transform to White-Balance
    img_new = np.zeros(img.shape)
    img_new[:,:,0] = scale_R * img_R  
    img_new[:,:,1] = scale_G * img_G 
    img_new[:,:,2] = scale_B * img_B  
    
    # Normalize Images
    max_intensity = np.max(np.max(np.max(img_new)))
    min_intensity = np.min(np.min(np.min(img_new)))
    
    img_normalized = (((img_new - min_intensity) / (max_intensity - min_intensity))*255).astype(np.uint8)

    # Illuminant Profile (Gray World Color Constancy) 
    illuminance = [avg_R, avg_G, avg_B]
    
    return img_normalized, illuminance
def extract_features(image,mask=None):    
    # Color Spaces: I/O ------------------------------------------------------------------------------------------------------------------------------------------------------
    img_RGB               = image #original image 
    img_GL                = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY) #convert to grey
    img_HSV               = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV) #convert to HSV

    circa_mask            = create_circular_mask(image.shape[0], image.shape[1], radius = 300).astype(bool) # create circular mask around ROI
    
    #create masked lesions for three color spaces using above function
    masked_lesion_GL      = np.ma.array(np.multiply(img_GL,    circa_mask)  ,mask=~circa_mask)
    masked_lesion_RGB     = np.ma.array(np.multiply(img_RGB,   np.dstack((circa_mask,circa_mask,circa_mask))), mask=~np.dstack((circa_mask,circa_mask,circa_mask)))
    masked_lesion_HSV     = np.ma.array(np.multiply(img_HSV,   np.dstack((circa_mask,circa_mask,circa_mask))), mask=~np.dstack((circa_mask,circa_mask,circa_mask)))
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    
    # Color Constancy Spaces: I/O ---------------------------------------------------------------------------------------------------------------------------------------------
    #applying color constancy to detect the color of image independent of the light source and amount of varying illumination and image aquistaion methods
    img_ccRGB,_           = color_constant(image)
    img_ccGL              = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY)
    img_ccHSV             = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)

    #masked lesions of CC 
    masked_lesion_ccGL    = np.ma.array(np.multiply(img_ccGL,    circa_mask)  ,mask=~circa_mask)
    masked_lesion_ccRGB   = np.ma.array(np.multiply(img_ccRGB,   np.dstack((circa_mask,circa_mask,circa_mask))), mask=~np.dstack((circa_mask,circa_mask,circa_mask)))
    masked_lesion_ccHSV   = np.ma.array(np.multiply(img_ccHSV,   np.dstack((circa_mask,circa_mask,circa_mask))), mask=~np.dstack((circa_mask,circa_mask,circa_mask)))

    #masked lesions of CC max rgb normlaization
    img_mxRGB             = (correct_image(image, grey_edge(image, njet=0, mink_norm=-1, sigma=0))*255).astype(np.uint8)
    img_mxGL              = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2GRAY)
    img_mxHSV             = cv2.cvtColor(img_RGB, cv2.COLOR_RGB2HSV)
    
    masked_lesion_mxGL    = np.ma.array(np.multiply(img_mxGL,    circa_mask)  ,mask=~circa_mask)
    masked_lesion_mxRGB   = np.ma.array(np.multiply(img_mxRGB,   np.dstack((circa_mask,circa_mask,circa_mask))), mask=~np.dstack((circa_mask,circa_mask,circa_mask)))
    masked_lesion_mxHSV   = np.ma.array(np.multiply(img_mxHSV,   np.dstack((circa_mask,circa_mask,circa_mask))), mask=~np.dstack((circa_mask,circa_mask,circa_mask)))
    #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
    
    
    # Color Moments ----------------------------------------------------------------------------------------------------------------------------------------------------------------
    #first order statistics of original image
    mean_R, std_R, skew_R, kurt_R, mean_G,  std_G,  skew_G,  kurt_G,  mean_B,  std_B,  skew_B,  kurt_B   = color_moments(masked_lesion_RGB,     channel=3)
    mean_H, std_H, skew_H, kurt_H, mean_S,  std_S,  skew_S,  kurt_S,  mean_V,  std_V,  skew_V,  kurt_V   = color_moments(masked_lesion_HSV,     channel=3)
    #first order stats in gray world constancy
    mean_ccR, std_ccR, skew_ccR, kurt_ccR, mean_ccG,  std_ccG,  skew_ccG,  kurt_ccG,  mean_ccB,  std_ccB,  skew_ccB,  kurt_ccB   = color_moments(masked_lesion_ccRGB,   channel=3)
    mean_ccH, std_ccH, skew_ccH, kurt_ccH, mean_ccS,  std_ccS,  skew_ccS,  kurt_ccS,  mean_ccV,  std_ccV,  skew_ccV,  kurt_ccV   = color_moments(masked_lesion_ccHSV,   channel=3)
    #first order stats in max rgb normalization
    mean_mxR, std_mxR, skew_mxR, kurt_mxR, mean_mxG,  std_mxG,  skew_mxG,  kurt_mxG,  mean_mxB,  std_mxB,  skew_mxB,  kurt_mxB   = color_moments(masked_lesion_mxRGB,   channel=3)
    mean_mxH, std_mxH, skew_mxH, kurt_mxH, mean_mxS,  std_mxS,  skew_mxS,  kurt_mxS,  mean_mxV,  std_mxV,  skew_mxV,  kurt_mxV   = color_moments(masked_lesion_mxHSV,   channel=3)
    #low-level statistics-based methods like Grey-world (GW), MAX-RGB and Grey-Edge (GE)
#     #-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


    # Graylevel Co-Occurrence Matrix -------------------------------------------------------------------------------------------------------------------------
    GLCM_RGB   = GLCM(masked_lesion_RGB,   channel=3)
    GLCM_HSV   = GLCM(masked_lesion_HSV,   channel=3)

    #----------------------------------------------------------------------------------------------------------------------------------------------------------

    
    # Color Markers -------------------------------------------------------------------------------------------------------------
    CM_black, CM_red, CM_bluegray, CM_white, CM_lightbrown, CM_darkbrown = melanoma_color_markers(masked_lesion_RGB, circa_mask)  
    #-------------------------------------------------------------------------------------------------------------------------------
    
    
    # Local Binary Patterns --------------------------------------------------------------------------------------
    lbp_R, lbp_G, lbp_B    = LBP(masked_lesion_RGB,   channel=3)
    lbp_H, lbp_S, lbp_V    = LBP(masked_lesion_HSV,   channel=3)
    
    
    LBP_CGLF  = np.concatenate((lbp_R,lbp_G,lbp_B,lbp_H,lbp_S,lbp_V),axis=0)
   
    #--------------------------------------------------------------------------------------------------------------
    
    
    #--------------------------------------------------------------------------------------------------------------    
    
               
             
    
    features = [ mean_R, std_R, skew_R, mean_G,  std_G,  skew_G,  mean_B,  std_B,  skew_B,   
                 mean_H, std_H, skew_H, mean_S,  std_S,  skew_S,  mean_V,  std_V,  skew_V,                  
                 CM_black,CM_red, CM_bluegray, CM_white, CM_lightbrown, CM_darkbrown,
                 mean_ccR, std_ccR, skew_ccR, mean_ccG,  std_ccG,  skew_ccG,  mean_ccB,  std_ccB,  skew_ccB, 
                 mean_ccH, std_ccH, skew_ccH, mean_ccS,  std_ccS,  skew_ccS,  mean_ccV,  std_ccV,  skew_ccV,                
                 mean_mxR, std_mxR, skew_mxR, mean_mxG,  std_mxG,  skew_mxG,  mean_mxB,  std_mxB,  skew_mxB, 
                 mean_mxH, std_mxH, skew_mxH, mean_mxS,  std_mxS,  skew_mxS,  mean_mxV,  std_mxV,  skew_mxV]
    features = np.concatenate((features, GLCM_RGB, GLCM_HSV, LBP_CGLF),axis=0)
    
    return features

def concat_and_shuffled(class0,class1, shuffled=True):
    rnd = np.random.RandomState(8)
    
    # Create Individual Label Vectors
    Y_0   = np.zeros(class0.shape[0])
    Y_1   = np.ones(class1.shape[0])
    
    # Concatenate to Complete Vectors
    Y     = np.concatenate([Y_0,Y_1])
    X     = np.concatenate([class0,class1])
    
    if shuffled:
        shuffled_indices = rnd.permutation(np.arange(Y.shape[0]))
        return X[shuffled_indices], Y[shuffled_indices]
    else:
        return X, Y
#############################################################################################################################################################



###############################################################################################################
############################################## Training Images ################################################
###############################################################################################################
train_dat = dict()
train_dat['feature'] = []
train_dat['label'] = []
temp=[]
data = pd.read_csv('/home/nisma/final_labels1.csv')
print(data.head())
for i in range(0,2000):#range(len(train_Array)): 
    img_o=cv2.imread(pathFolder_train+train_Array[i])
    print(train_Array[i])
    img_seg,img_r = segmentation(img_o)
    
    h1,w1,c1=img_seg.shape
    if img_seg[int(h1/2),int(w1/2)].all()==0:
       img_seg=255-img_seg
    
    doctor = cv2.bitwise_and(img_r, img_seg, mask=None)
    #resized
    resized_up=cv2.resize(doctor, (img_o.shape[1],img_o.shape[0]), interpolation= cv2.INTER_AREA)
    #print(doctor[24].shape)

    cv2.imwrite(os.path.join(outputFolder_path+train_Array[i] ),resized_up)
    features = extract_features(resized_up)
    #converting features to list
    m_list = features.tolist()
    file = open('features.csv1', 'a', newline ='') 
    with file:     
        write = csv.writer(file) 
        write.writerow(m_list) 

    if (data.loc[[i],["melanoma"]].values==1):
            train_dat['label'].append(1)
            temp.append(features)
    if (data.loc[[i],["seborrheic_keratosis"]].values==1):
            train_dat['label'].append(2)
            
            temp.append(features)
    if (data.loc[[i],["benign"]].values==1):
            train_dat['label'].append(0)
            temp.append(features)

data_all = pd.DataFrame({"Features": temp, "category": train_dat['label']})
print(data_all.head())
###########################################################################################
###########################################################################################
#pdb.set_trace()
X = np.array(temp)
y = np.array(data_all["category"])
#  considering our we apply to data.
#Firstly, I specify the random forest instance, indicating the number of trees.
#Then I use selectFromModel object from sklearn to automatically select the features.
# SelectFromModel is a meta-transformer that can be used alongside any estimator that assigns importance to each feature through a specific attribute
#the unimportant features are removed according to threshold.
#SelectFromModel will select those features which importance is greater than the mean importance of all the features by default, but we can alter this threshold if we want.
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X, y)
# To see which features are important we can use get_support method on the fitted model.
selected_feat= X[ :,(sel.get_support())]
print(X.shape)
#It will return an array of boolean values. True for the features whose importance is greater than the mean importance and False for the rest.
print(selected_feat.shape)
X_train, X_test, y_train, y_test = train_test_split(selected_feat, y, test_size=0.33, random_state=42)


# some time later...
# load the model from disk

'''loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)
'''

#Can also use SVM but RF is faster and may be more accurate.
#from sklearn import svm

###############################################################

model = KNeighborsClassifier(n_neighbors=2,
                             n_jobs=1)

model.fit(X_train, y_train)

accuracy = accuracy_score(model.predict(X_train), y_train)
print(accuracy)

n_neighbors = np.array([7,8,9,10,12,15,20])
param_grid = dict(n_neighbors=n_neighbors)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_.n_neighbors)
y_pred1=grid.predict(X_test)
print(classification_report(y_test, y_pred1))
b=balanced_accuracy_score(y_test, y_pred1)
print("balanced knn",b)

######################################################RF#####################################
#RandomizedSearchCV implements a “fit” method and a “predict” method like any classifier except that the parameters of the classifier used to predict is optimized by cross-validation.
n_estimators      = [int(x) for x in np.linspace(start=70, stop=250, num=25)]
max_features      = ['auto', 'sqrt']
max_depth         = [int(x) for x in np.linspace(start=5, stop=100, num=10)]
min_samples_leaf  = [1, 2, 5, 10]
min_samples_split = [2, 5, 10, 20]
bootstrap         = [False]
max_depth.append(None)
random_grid = {'n_estimators': n_estimators, 'max_features': max_features, 'max_depth': max_depth,
                   'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}
grid_search = RandomizedSearchCV(RandomForestClassifier(class_weight='balanced', random_state=0),
                                     random_grid, n_iter=100, cv=5, random_state=0, n_jobs=-1)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
y_pred2=grid_search.predict(X_test)
print(classification_report(y_test, y_pred2))
b=balanced_accuracy_score(y_test, y_pred)
print("balanced rf",b)
############################################################SVM#####################################
n_estimators      = [int(x) for x in np.linspace(start=70, stop=250, num=25)]
max_features      = ['auto', 'sqrt']
max_depth         = [int(x) for x in np.linspace(start=5, stop=100, num=10)]
min_samples_leaf  = [1, 2, 5, 10]
min_samples_split = [2, 5, 10, 20]
bootstrap         = [False]
max_depth.append(None)
random_grid = {'n_estimators': n_estimators, 'max_features': max_features, 'max_depth': max_depth,
                   'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf, 'bootstrap': bootstrap}
grid_search = RandomizedSearchCV(RandomForestClassifier(class_weight='balanced', random_state=0),
                                     random_grid, n_iter=100, cv=5, random_state=0)
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)
y_pred2=grid_search.predict(X_test)
print(classification_report(y_test, y_pred2))
b=balanced_accuracy_score(y_test, y_pred2)
print("balanced rf",b)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
